In [ ]:
import utils as my
import pandas as pd
import pubchempy as pbp
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import torch
import numpy as np

In [ ]:
data = pd.read_csv('../data.csv')
smiles = data['smiles']
CI_raw = data['A2780 CS']
smiles_nonan = []
CI = []
for i in range(len(smiles)):
    if not pd.isna(smiles[i]) and not pd.isna(CI_raw[i]):
        smiles_nonan.append(smiles[i])
        CI.append(my.strtonp(CI_raw[i]))
CI = np.asarray(CI)

In [ ]:
# fingerprints = my.get_compounds(smiles_nonan, result='fingerprint', identifier='smiles')

In [ ]:
fingerprints = []
for i in smiles_nonan:
    molecular = Chem.MolFromSmiles(i)
    fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(molecular, 6, nBits=1024)
    fingerprint = list(fingerprint)
    fingerprints.append(fingerprint)


In [29]:
features = torch.tensor(fingerprints, dtype=torch.double)
labels = torch.tensor(CI[:,1])
# data_processed = pd.DataFrame({'features': features, 'labels': labels})
print(features.shape)

torch.Size([341, 1024])
